# Image inpainting

Example of image inpainting using NIHT applied to nearfield acoustic holography data.

First load packages

In [ ]:
import sys
sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import cv2

import pylops

import scipy.io as sio

from nopt.transforms import *
from nopt.constraints import *
from nopt.problems import *
from nopt.solvers import *

Now load the (complex) data

In [ ]:
# Load image
mat_contents = sio.loadmat('../data/PressureZ0.mat')
u0: np.float64 = 0.1155
c0: np.float64 = 1500.0
rho0: np.float64 = 1000.0
p0: np.float64 = u0 * c0 * rho0
img: np.ndarray = p0 * np.conj(mat_contents['out'])

delta: float = 0.5
rho: float = 0.1
p: int = round(delta * 512**2) # number of samples
s: int = round(p * rho) # sparsity

mask = np.random.choice(512**2, p, replace=False)

P_omega = EntryWise(mask, 512**2)

Wave2d = pylops.signalprocessing.DWT2D(img.shape, level=2, wavelet = 'db4')

img_coeff = Wave2d.matvec(img.flatten())

The model is:

$$ 
\min_c \| P_\Omega( \Psi^*(c) )) - b \|_2 \qquad \mathrm{such that} \qquad \|c\|_0 \leq s
$$

where $P_\Omega$ is an entrywise subsampling and $\Psi$ is an 2D Wavelet transform.

In [ ]:
A = CompositeTransform([Wave2d.adjoint(), P_omega])
HTs = Sparsity(s)
b = P_omega.matvec(img.flatten())

sub, x_true = HTs.project(img_coeff)
problem = LinearProblem(A, b, HTs, x_true = x_true)
solver = NIHT(logverbosity = 2, maxiter = 100, verbosity = 2, minreldecrease = 1-1e-3)

Now solve:

In [ ]:
x, opt_log = solver.solve(problem)

and plot

In [ ]:
fig, axs = plt.subplots(1,2, dpi=180)
fig.suptitle('Subsampled image and the recovered image')
plt1 = axs[0].imshow(P_omega.rmatvec(b).reshape(512,512), vmin = 0, vmax=256)
plt2 = axs[1].imshow(Wave2d.rmatvec(x).reshape(512,512), vmin = 0, vmax=256)
plt.show()

The convergence is:

In [ ]:
plt.plot(opt_log['iterations']['fx'])